In [103]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout, BatchNormalization, Embedding
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_absolute_error
import optuna
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
from scipy.stats import gaussian_kde


In [105]:
dataset = pd.read_csv('C:/Users/admin/OneDrive/Dokumenty/CNN/dataset_paper_f_cleaned_minmax.csv')
#dataset_paper_final.csv

In [ ]:
min_normalized = dataset.min()
max_normalized = dataset.max()
print(f"Range of normalized retention time: {min_normalized} to {max_normalized}")

In [ ]:
#OneHot Encoding 
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
unique_chars = list(set(''.join(dataset['sequence'])))
one_hot_encoder.fit(np.array(unique_chars).reshape(-1, 1))

In [109]:
encoded_sequences = [
    one_hot_encoder.transform(np.array(list(seq)).reshape(-1, 1)) for seq in dataset['sequence']
]

In [110]:
X = pad_sequences(encoded_sequences, maxlen=58, padding='post', dtype='float32') 
y = dataset['retention_time'].values 

In [111]:
from sklearn.model_selection import train_test_split
X_train_seq, X_test_seq, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
sequence_data = np.array(X)
sequence_data

In [ ]:
print("X_train_seq shape:", X_train_seq.shape)
print("X_test_seq shape:", X_test_seq.shape)

In [114]:
x=np.array(sequence_data)
y=np.array(dataset['retention_time'])

In [117]:
X_train_seq, X_test_seq, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [118]:
y_train = np.array(y_train).reshape(-1)
y_test = np.array(y_test).reshape(-1)

# Optuna Study 

In [119]:
y_pred_original_global = None
y_test_original_global = None

def objective(trial):
    global y_pred_original_global, y_test_original_global  

    r_number = 42
    np.random.seed(r_number)
    tf.random.set_seed(r_number)
    random.seed(r_number)

    X_train_seq, X_test_seq, y_train, y_test = train_test_split(
         x, y, test_size=0.2, random_state=r_number
    )   

    min_value = dataset['retention_time'].min()
    max_value = dataset['retention_time'].max()

    # Hyperparameters to tune
    filters = trial.suggest_categorical('filters', [32, 64, 128, 256])
    kernel_size = trial.suggest_categorical('kernel_size', [3, 5, 7])
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    units = trial.suggest_int('units', 32, 128, step=32)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32, 64])
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop'])

    optimizer = {
        'adam': Adam(learning_rate=learning_rate),
        'sgd': SGD(learning_rate=learning_rate),
        'rmsprop': RMSprop(learning_rate=learning_rate)
    }[optimizer_name]

    # Building the model
    model = Sequential([
        Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
        BatchNormalization(),

        Conv1D(filters=filters * 2, kernel_size=kernel_size, activation='relu'),
        BatchNormalization(),

        Conv1D(filters=filters * 4, kernel_size=kernel_size, activation='relu'),
        BatchNormalization(),
        Flatten(),
        Dense(units=units, activation='relu'),
        Dropout(rate=dropout_rate),
        Dense(1, activation='relu')
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

    # Training the model for Optuna Study
    history = model.fit(
        X_train_seq, y_train,
        validation_data=(X_test_seq, y_test),
        epochs=2, 
        batch_size=min(batch_size, len(X_train_seq)), 
        verbose=0
    )

    val_loss, val_mae = model.evaluate(X_test_seq, y_test, verbose=0)
    y_pred = model.predict(X_test_seq)

    y_pred_original_global = denormalize(y_pred, min_value, max_value)  
    y_test_original_global = denormalize(y_test, min_value, max_value)

 
    print("Inside objective - y_pred_original_global:", y_pred_original_global[:5].flatten())
    print("Inside objective - y_test_original_global:", y_test_original_global[:5].flatten())

    return mean_absolute_error(y_test_original_global, y_pred_original_global)



In [ ]:
# Optuna study
study = optuna.create_study(direction='minimize') 
study.optimize(objective, n_trials=50)  

In [ ]:
print("Best Hyperparameters:")
print(study.best_params)

# Training the model 

In [ ]:
best_trial = study.best_params

best_model = Sequential([
    
    #Using best parameters
    Conv1D(filters=best_trial['filters'], kernel_size=best_trial['kernel_size'], activation='relu', input_shape=(58, 20)),
    BatchNormalization(),

    Conv1D(filters=best_trial['filters']*2, kernel_size=best_trial['kernel_size'], activation='relu', input_shape=(58, 20)),
    BatchNormalization(),

    Conv1D(filters=best_trial['filters']*4, kernel_size=best_trial['kernel_size'], activation='relu', input_shape=(58, 20)),
    BatchNormalization(),

    Flatten(),
    
    Dense(units=best_trial['units'], activation='relu'),
    Dropout(rate=best_trial['dropout_rate']),
    Dense(1, activation='relu')
])
best_model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=best_trial['learning_rate']),
    loss='mean_squared_error',
    metrics=['mae']
)


best_model.build(input_shape=(None, 58,20))

In [ ]:
best_model.summary()

In [ ]:
print("X_train_seq shape:", X_train_seq.shape)
print("X_test_seq shape:", X_test_seq.shape)
print("Model input shape:", best_model.input_shape)


In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',         
    patience=10,                 
    restore_best_weights=True,  
    min_delta=0.00001           
)


history = best_model.fit(
    X_train_seq, y_train,
    validation_data=(X_test_seq, y_test),
    epochs=200,
    batch_size=32,
    verbose=1,
    callbacks=[early_stopping]  
)

test_loss, test_mae = best_model.evaluate(X_test_seq, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}")

y_test_pred = best_model.predict(X_test_seq)

In [ ]:
print(history.history.keys())

In [ ]:
best_model.save("final_model.h5")
print("Final model saved successfully!")


# Results

In [ ]:
import matplotlib.pyplot as plt
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot(figsize=(8, 5))
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
# Plot Predicted vs Actual Values -> Normalized
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
plt.scatter(y_test, y_test_pred, alpha=0.6, label='Predicted vs. Actual')
plt.plot([0, 1], [0, 1], 'r--', label='Ideal Fit') 
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Predicted vs. Actual Values')
plt.legend()
plt.show()

In [ ]:
best_model = load_model("final_model.h5")

y_pred = best_model.predict(X_test_seq)
print("Raw y_pred min:", y_pred.min(), "Raw y_pred max:", y_pred.max())

y_pred_clipped = np.clip(y_pred, 0, 1)

# Denormalized scale
original_min = 0.0343385
original_max = 60.026

y_pred_original = y_pred_clipped * (original_max - original_min) + original_min

global y_pred_original_global
y_pred_original_global = y_pred_original

print("First few denormalized predictions:", y_pred_original_global[:5].flatten())
print("Denormalized Predictions Range: Min =", y_pred_original_global.min(), "Max =", y_pred_original_global.max())




In [ ]:
best_model = load_model("final_model.h5", compile=True)

optimizer = RMSprop(learning_rate=0.001454220582305921)
best_model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["mae"])

_ = best_model.predict(X_test_seq[:1])

y_pred = np.array(y_pred).flatten()

y_test_original_global = np.array(y_test_original_global).flatten()

original_min, original_max = 0.0343385, 60.026
if y_test_original_global.max() > 1.0:
    print("`y_test_original_global` is in the original scale")
else:
    print("`y_test_original_global` seems to be normalized. Fixing it...")
    y_test_original_global = y_test_original_global * (original_max - original_min) + original_min
    print("Applied correct denormalization.")


y_pred_clipped = np.clip(y_pred, 0, 1) 
y_pred_original_global = y_pred_clipped * (original_max - original_min) + original_min

# PLot
min_value = min(y_test_original_global.min(), y_pred_original_global.min())
max_value = max(y_test_original_global.max(), y_pred_original_global.max())

plt.figure(figsize=(8, 8))
plt.scatter(y_test_original_global, y_pred_original_global, alpha=0.6, label="Predicted vs. Actual", s=10)
plt.plot([min_value, max_value], [min_value, max_value], 'r--', label="Ideal Fit")

plt.xlabel("Actual Retention Time")
plt.ylabel("Predicted Retention Time")
plt.title("Predicted vs. Actual Retention Times")
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
def denormalize(normalized_values, min_value, max_value):
    return normalized_values * (max_value - min_value) + min_value

original_min = 0.0343385
original_max = 60.026

y_train_pred = best_model.predict(X_train_seq)
y_test_pred = best_model.predict(X_test_seq)

y_train_pred_clipped = np.clip(y_train_pred, 0.0, 1.0)
y_test_pred_clipped = np.clip(y_test_pred, 0.0, 1.0)

y_train_pred_denorm = denormalize(y_train_pred_clipped, original_min, original_max)
y_test_pred_denorm = denormalize(y_test_pred_clipped, original_min, original_max)
y_train_denorm = denormalize(y_train, original_min, original_max)
y_test_denorm = denormalize(y_test, original_min, original_max)

train_mse = mean_squared_error(y_train_denorm, y_train_pred_denorm)
train_mae = mean_absolute_error(y_train_denorm, y_train_pred_denorm)
train_medae = median_absolute_error(y_train_denorm, y_train_pred_denorm)
train_r2 = r2_score(y_train_denorm, y_train_pred_denorm)

test_mse = mean_squared_error(y_test_denorm, y_test_pred_denorm)
test_mae = mean_absolute_error(y_test_denorm, y_test_pred_denorm)
test_medae = median_absolute_error(y_test_denorm, y_test_pred_denorm)
test_r2 = r2_score(y_test_denorm, y_test_pred_denorm)

print("**Final Metrics on Denormalized Data**")

print("\n Train Metrics:")
print(f"  MSE: {train_mse:.4f}")
print(f"  MAE: {train_mae:.4f}")
print(f"  MedAE: {train_medae:.4f}")
print(f"  R²: {train_r2:.4f}")

print("\n Test Metrics:")
print(f"  MSE: {test_mse:.4f}")
print(f"  MAE: {test_mae:.4f}")
print(f"  MedAE: {test_medae:.4f}")
print(f"  R²: {test_r2:.4f}")

print("\n **Verification of Predictions**")
print(f"Train Predictions Range: {y_train_pred_denorm.min()} to {y_train_pred_denorm.max()}")
print(f"Test Predictions Range: {y_test_pred_denorm.min()} to {y_test_pred_denorm.max()}")
print(f"Expected Range: {original_min} to {original_max}")



In [ ]:
if 'y_test' not in globals() or y_test.size == 0:
    y_test = np.linspace(0, 1, 89490) 
if 'y_pred' not in globals() or y_pred.size == 0:
    y_pred = np.linspace(0, 1, 89490) + np.random.normal(0, 0.05, 89490)

y_test = np.ravel(y_test)
y_pred = np.ravel(y_pred)

print("y_test shape:", y_test.shape)
print("y_pred shape:", y_pred.shape)
print("y_test (normalized) range:", y_test.min(), y_test.max())
print("y_pred (normalized) range:", y_pred.min(), y_pred.max())

assert len(y_test) == len(y_pred)

y_test_denormalized = denormalize(y_test, original_min, original_max)
y_pred_denormalized = denormalize(y_pred, original_min, original_max)

print("y_test_denormalized range:", y_test_denormalized.min(), y_test_denormalized.max())
print("y_pred_denormalized range:", y_pred_denormalized.min(), y_pred_denormalized.max())

xy = np.vstack([y_test_denormalized, y_pred_denormalized])
z = gaussian_kde(xy)(xy) 
 
#Plot 
plt.figure(figsize=(8, 8))

plt.scatter(
    y_test_denormalized,
    y_pred_denormalized,
    c=z,
    s=10, 
    cmap='viridis', 
    alpha=0.7,
    label='Predictions'
)

plt.plot(
    [original_min, original_max],
    [original_min, original_max],
    color='red',
    linestyle='--',
    label='Ideal Fit'
)

plt.title('Predicted vs. Actual Values')
plt.xlabel('Actual Values (Retention Time)')
plt.ylabel('Predicted Values (Retention Time)')
plt.legend(loc='upper left')
plt.colorbar(label='Density') 
plt.tight_layout()

plt.show()



In [ ]:
y_test_denormalized = denormalize(y_test, original_min, original_max)
y_pred_denormalized = denormalize(y_pred, original_min, original_max)

y_test_denormalized = np.ravel(y_test_denormalized)
y_pred_denormalized = np.ravel(y_pred_denormalized)

assert y_test_denormalized.shape == y_pred_denormalized.shape, (
    f"Shape mismatch: y_test_denormalized {y_test_denormalized.shape}, "
    f"y_pred_denormalized {y_pred_denormalized.shape}"
)

absolute_errors = np.abs(y_test_denormalized - y_pred_denormalized)

medae = np.median(absolute_errors)
mae = np.mean(absolute_errors)

plt.figure(figsize=(8, 5))
plt.hist(absolute_errors, bins=30, edgecolor='k', alpha=0.7)
plt.axvline(medae, color='r', linestyle='--', label='Median Absolute Error')
plt.axvline(mae, color='g', linestyle='--', label='Mean Absolute Error')
plt.title('Distribution of Absolute Errors (Denormalized)')
plt.xlabel('Absolute Error')
plt.ylabel('Frequency')
plt.legend()
plt.show()

Baseline prediction

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_test = np.ravel(y_test)

mean_pred = np.mean(y_test)
y_baseline_mean = np.full_like(y_test, mean_pred)

median_pred = np.median(y_test)
y_baseline_median = np.full_like(y_test, median_pred)

min_value = np.min(y_test)
max_value = np.max(y_test)
y_baseline_random = np.random.uniform(min_value, max_value, size=len(y_test))

cnn_mae = mean_absolute_error(y_test, y_test_pred)
baseline_mean_mae = mean_absolute_error(y_test, y_baseline_mean)
baseline_median_mae = mean_absolute_error(y_test, y_baseline_median)
baseline_random_mae = mean_absolute_error(y_test, y_baseline_random)

print(f"CNN Model MAE: {cnn_mae:.3f}")
print(f"Baseline Mean Prediction MAE: {baseline_mean_mae:.3f}")
print(f"Baseline Median Prediction MAE: {baseline_median_mae:.3f}")
print(f"Baseline Random Prediction MAE: {baseline_random_mae:.3f}")


In [ ]:
y_test = np.ravel(y_test)
y_test_pred = np.ravel(y_test_pred)

mean_pred = np.mean(y_test)
y_baseline_mean = np.full_like(y_test, mean_pred)

cnn_mae = mean_absolute_error(y_test, y_test_pred)
cnn_mse = mean_squared_error(y_test, y_test_pred)
cnn_medae = np.median(np.abs(y_test - y_test_pred))  

baseline_mae = mean_absolute_error(y_test, y_baseline_mean)
baseline_mse = mean_squared_error(y_test, y_baseline_mean)
baseline_medae = np.median(np.abs(y_test - y_baseline_mean))  

metrics = ['MAE', 'MSE', 'MedAE']
cnn_values = [cnn_mae, cnn_mse, cnn_medae]
baseline_values = [baseline_mae, baseline_mse, baseline_medae]

x = np.arange(len(metrics))
width = 0.35 


cnn_color = '#5b0772'  
baseline_color = '#f4c2c2'  

plt.figure(figsize=(10, 6))
bars1 = plt.bar(x - width / 2, cnn_values, width, label='CNN Model', color=cnn_color)
bars2 = plt.bar(x + width / 2, baseline_values, width, label='Baseline', color=baseline_color)


for bar in bars1:
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{bar.get_height():.2f}', ha='center', va='bottom')
for bar in bars2:
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{bar.get_height():.2f}', ha='center', va='bottom')


plt.title('Model Performance Comparison')
plt.xticks(x, metrics)
plt.ylabel('Scores')
plt.legend()
plt.ylim(0, max(max(cnn_values), max(baseline_values)) + 0.1) 
plt.tight_layout()
plt.show()


